# Actividad - Proyecto práctico (Versión con entorno SpaceInvaders y CNN)

> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Granizo, Mateo
*   Alumno 2: Maiolo, Pablo
*   Alumno 3: Miglino, Diego
---
## **PARTE 1** - Instalación y requisitos previos

> NOTA: Esta versión usa el entorno SpaceInvaders-v0 y una CNN para mejorar el rendimiento.

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive in the case of estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.
---
### 1.1. Preparar enviroment (solo local)

> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. Instalar Anaconda
2. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.

```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
pip install jupyter
pip install atari-py
```

3. Abrir la notebook con *jupyter-notebook*.

```
jupyter-notebook
```


In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/TP_Grupal"
mount='./'

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False
print(IN_COLAB)

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
# ejecutar solo la primera vez..
# Esta versión no requiere atari-py, solo gym básico

if IN_COLAB:
  %pip install gym==0.17.3
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12
  %pip install atari-py
else:
  %pip install gym==0.17.3
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install atari-py

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.
  - NOTA: Hemos cambiado el entorno para usar SpaceInvaders-v0 y una CNN.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de X puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [ ]:
from __future__ import division
import numpy as np
import gym
import tensorflow as tf
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint, Callback
from rl.core import Processor

#### Configuración base

In [ ]:
# Usamos SpaceInvaders-v0
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

print("Numero de acciones disponibles: " + str(nb_actions))
print("Formato de las observaciones: " + str(env.observation_space))

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # Convertir a escala de grises
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # Guardar como uint8 para ahorrar memoria

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.  # Normalizar a [0, 1]
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)  # Recortar recompensas para estabilizar el aprendizaje

1. Implementación de la red neuronal (CNN)

In [ ]:
# Modelo CNN para DQN
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
if K.image_data_format() == 'channels_last':
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')
model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))
print(model.summary())

2. Implementación de la solución DQN

In [ ]:
# Configuración del agente DQN
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1.0, value_min=0.1, value_test=0.01,
                              nb_steps=100000)

dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy,
               memory=memory, processor=processor,
               nb_steps_warmup=10000, gamma=0.99, train_interval=4, delta_clip=1.0)
dqn.compile(Adam(learning_rate=1e-4), metrics=['mae'])

In [ ]:
# Entrenamiento del agente
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=10000)]
callbacks += [FileLogger(log_filename, interval=1000)]

# Entrenamos por 200,000 pasos
dqn.fit(env, callbacks=callbacks, nb_steps=200000, log_interval=10000, visualize=False)

# Guardamos los pesos finales
dqn.save_weights(weights_filename, overwrite=True)

In [ ]:
# Test de n episodios para calcular la recompensa final
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=True)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

### Justificación de los parámetros seleccionados

En esta implementación, se han realizado varias modificaciones importantes para mejorar el rendimiento del agente DQN en el entorno SpaceInvaders-v0:

1. **Arquitectura de la red neuronal (CNN)**:
   - Se ha utilizado una red neuronal convolucional (CNN) para capturar las características visuales del entorno SpaceInvaders-v0.
   - La CNN consta de varias capas convolucionales, capas de pooling y capas fully connected.
   - Esta arquitectura es más adecuada para entornos basados en imágenes que las redes neuronales densas utilizadas anteriormente.

2. **Preprocesamiento de las imágenes (AtariProcessor)**:
   - Se ha utilizado un procesador para preprocesar las imágenes del entorno antes de pasarlas a la red neuronal.
   - El preprocesamiento incluye el cambio de tamaño de las imágenes, la conversión a escala de grises y la normalización de los valores de los píxeles.
   - Esto ayuda a reducir la complejidad del problema y a mejorar la eficiencia del aprendizaje.

3. **Aumento del tiempo de entrenamiento**:
   - Se ha aumentado el tiempo de entrenamiento a 200,000 pasos para permitir que el agente aprenda una política más óptima.

4. **Ajuste de los hiperparámetros**:
    - Se ha ajustado la política de exploración, reduciendo el valor de `value_test` a 0.01 para reducir la exploración durante la fase de test.
    - Se ha reducido la tasa de aprendizaje a 1e-4 para estabilizar el aprendizaje.

### Resultados esperados

Con estos parámetros, esperamos que el agente logre una puntuación media significativamente mayor en el modo test en el entorno SpaceInvaders-v0. La CNN debería permitir al agente aprender características visuales importantes del juego, lo que le permitirá tomar decisiones más informadas y obtener mejores resultados.

---